<img src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png" align=left alt="CC BY license logo" /><br /><br />
Created by [Nathan Kelber](http://nkelber.com) and Ted Lawless for [JSTOR Labs](https://labs.jstor.org/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
**For questions/comments/improvements, email nathan.kelber@ithaka.org.**<br />
____

# Finding Significant Words Using TF/IDF

**Description:**
This [notebook](https://constellate.org/docs/key-terms/#jupyter-notebook) shows how to discover significant words. The method for finding significant terms is [tf-idf](https://constellate.org/docs/key-terms/#tf-idf).  The following processes are described:

* An educational overview of TF-IDF, including how it is calculated
* Using the `constellate` client to retrieve a dataset
* Filtering based on a pre-processed ID list
* Cleaning the tokens in the dataset
* Creating a [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary)
* Creating a [gensim](https://constellate.org/docs/key-terms/#gensim) [bag of words](https://constellate.org/docs/key-terms/#bag-of-words) [corpus](https://constellate.org/docs/key-terms/#corpus)
* Computing the most significant words in your [corpus](https://constellate.org/docs/key-terms/#corpus) using [gensim](https://constellate.org/docs/key-terms/#gensim) implementation of [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf)

**Use Case:** For Learners (Detailed explanation, not ideal for researchers)

**Difficulty:** Intermediate

**Completion time:** 60 minutes

**Knowledge Required:** 
* Python Basics Series ([Start Python Basics I](./python-basics-1.ipynb))

**Knowledge Recommended:**
* [Exploring Metadata](./exploring-metadata.ipynb)
* [Python Intermediate 2](./python-intermediate-2.ipynb)
* [Pandas I](./pandas-1.ipynb)
* A familiarity with [gensim](https://constellate.org/docs/key-terms/#gensim) is helpful but not required.

**Data Format:** [JSON Lines (.jsonl)](https://constellate.org/docs/key-terms/#jsonl)

**Libraries Used:**
* [constellate](https://constellate.org/docs/key-terms/#tdm-client) client to collect, unzip, and read our dataset
* [pandas](https://constellate.org/docs/key-terms/#pandas) to load a preprocessing list
* [gensim](https://constellate.org/docs/key-terms/#gensim) to help compute the [tf-idf](https://constellate.org/docs/key-terms/#tf-idf) calculations
* [NLTK](https://constellate.org/docs/key-terms/#nltk) to create a stopwords list (if no list is supplied)

**Research Pipeline:**

1. Build a dataset
2. Create a "Pre-Processing CSV" with [Exploring Metadata](./exploring-metadata.ipynb) (Optional)
3. Complete the TF-IDF analysis with this notebook
____

In [ ]:
# Import Modules and Libraries
import constellate
from collections import defaultdict
from pathlib import Path
import gensim

## What is "Term Frequency- Inverse Document Frequency" (TF-IDF)?

[TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) is used in [machine learning](https://constellate.org/docs/key-terms/#machine-learning) and [natural language processing](https://constellate.org/docs/key-terms//#nlp) for measuring the significance of particular terms for a given document. It consists of two parts that are multiplied together:

1. Term Frequency- A measure of how many times a given word appears in a document
2. Inverse Document Frequency- A measure of how many times the same word occurs in other documents within the corpus

**Before starting this lesson,** we recommend reading the [explanation of TF/IDF](https://constellate.org/docs/what-is-tf-idf) in the Constellate documentation.


### TF-IDF Calculation in Plain English

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

There are variations on the [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) formula, but this is the most widely-used version.

## Import your dataset

The next code cell tries to import your dataset trying each of the following methods:

1. Search for a custom dataset in the data folder 
2. Download a full dataset that has been requested
3. Download a sampled dataset (1500 items) that builds automatically when a dataset is created

If you are using a [dataset ID](https://constellate.org/docs/key-terms/#dataset-ID), replace the default dataset ID in the next code cell.

If you don't have a dataset ID, you can:
* Use the sample dataset ID already in the code cell
* [Create a new dataset](https://constellate.org/builder)
* [Use a dataset ID from other pre-built sample datasets](https://constellate.org/dataset/dashboard)

The Constellate client will download datasets automatically using either the `.download()` or `.get_dataset()` method.
* Full datasets are downloaded using the `.download()` method. They must be requested first in the builder environment. See the [Constellate client documentation](https://constellate.org/docs/constellate-client).

* Sampled datasets (1500 items) are downloaded using the `.get_dataset()` method. They are built automatically when a dataset is created.

In [ ]:
dataset_id = "7e41317e-740f-e86a-4729-20dab492e925"

# Check if a data folder exists. If not, create it.
data_folder = Path('./data/')
data_folder.mkdir(exist_ok=True)

# Check to see if a dataset file exists
# If not, download a dataset using the Constellate Client
# The default dataset is Shakespeare Quarterly, 1950-present
dataset_file = Path.cwd() / 'data' / 'my_data.jsonl.gz' # Make sure this filepath matches your dataset filename

if dataset_file.exists() == False:
    try: 
        dataset_file = constellate.download(dataset_id, 'jsonl')
    except: 
        dataset_file = constellate.get_dataset(dataset_id)

## Define a Unigram Processing Function
In this step, we gather the unigrams. If there is a Pre-Processing Filter, we will only analyze documents from the filtered ID list. We will also process each unigram, assessing them individually. We will complete the following tasks:

* Lowercase all tokens
* Remove tokens in stopwords list
* Remove tokens with fewer than 4 characters
* Remove tokens with non-alphabetic characters

We can define this process in a function.

In [ ]:
# Define a function that will process individual tokens
# Only a token that passes through all three `if` 
# statements will be returned. A `True` result for
# any `if` statement does not return the token. 

def process_token(token):
    token = token.lower()
    if len(token) < 4: # If True, do not return token
        return None
    if not(token.isalpha()): # If True, do not return token
        return None
    return token # If all are False, return the lowercased token

## Collect lists of Document IDs, Titles, and Unigrams

Next, we process all the unigrams into a list called `documents`. For demonstration purposes, this code runs on a limit of 500 documents, but we can change this to process all the documents. We are also collecting the document titles and ids so we can reference them later.

In [ ]:
documents = [] # A list that will contain all of our unigrams
document_ids = [] # A list that will contain all of our document ids
document_titles = [] # A list that will contain all of our titles

for document in constellate.dataset_reader(dataset_file):
    processed_document = [] # Temporarily store the unigrams for this document
    document_id = document['id'] # Temporarily store the document id for this document
    document_title = document['title'] # Temporarily store the document title for this document
    unigrams = document.get("unigramCount", [])
    for gram, count in unigrams.items():
        clean_gram = process_token(gram)
        if clean_gram is None:
            continue
        processed_document += [clean_gram] * count # Add the unigram as many times as it was counted
    if len(processed_document) > 0:
        document_ids.append(document_id)
        document_titles.append(document_title)
        documents.append(processed_document)

At this point, we have unigrams collected for all our documents insde the `documents` list variable. Each index of our list is a single document, starting with `documents[0]`. Each document is, in turn, a list with a single stringe for each unigram.

**Note:** As we collect the unigrams for each document, we are simply including them in a list of strings. This is not the same as collecting them into word counts, and we are not using a Counter() object here like the Word Frequencies notebook. 

The next cell demonstrates the contents of each item in our `document` list. Essentially, 

In [ ]:
# Show the unigrams collected for a particular document
# Change the value of n to see a different document
n = 0

print(document_titles[n])
list(documents[n])

If we wanted to see word frequencies, we could convert the lists at this point into `Counter()` objects. The next cell demonstrates that operation.

In [ ]:
# Convert a given document into a Counter object to determine
# word frequencies count

# Import counter to help count word frequencies
from collections import Counter

word_freq = Counter(documents[0]) # Change documents index to see a different document
word_freq.most_common(25) 

Now that we have all the cleaned unigrams for every document in a list called `documents`, we can use Gensim to compute TF/IDF.

---
## Using Gensim to Compute "Term Frequency- Inverse Document Frequency"

It will be helpful to remember the basic steps we did in the explanatory [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) example:

1. Create a list of the frequency of every word in every document
2. Create a list of every word in the [corpus](https://constellate.org/docs/key-terms/#corpus)
3. Compute [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) based on that data

So far, we have completed the first item by creating a list of the frequency of every word in every document. Now we need to create a list of every word in the corpus. In [gensim](https://constellate.org/docs/key-terms/#gensim), this is called a "dictionary". A [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) is similar to a [Python dictionary](https://constellate.org/docs/key-terms/#python-dictionary), but here it is called a [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) to show it is a specialized kind of dictionary.

### Creating a Gensim Dictionary

Let's create our [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary). A [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) is a kind of masterlist of all the words across all the documents in our corpus. Each unique word is assigned an ID in the gensim dictionary. The result is a set of key/value pairs of unique tokens and their unique IDs.

In [ ]:
dictionary = gensim.corpora.Dictionary(documents)

The [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) stores a unique identifier (starting with 0) for every unique token in the corpus. The [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) does not contain information on word frequencies; it only catalogs all the unique words in the corpus. You can see the unique ID for each token in the text using the .token2id() method.

In [ ]:
list(dictionary.token2id.items())

We could also look up the corresponding ID for a token using the ``.get`` method.

In [ ]:
# Get the value for the key 'people'. Return 0 if there is no token matching 'people'. 
# The number returned is the gensim dictionary ID for the token. 

dictionary.token2id.get('people', 0)

For the sake of example, we could also discover a particular token using just the ID number. This is not something likely to happen in practice, but it serves here as a demonstration of the connection between tokens and their ID number.

Normally, [Python dictionaries](https://constellate.org/docs/key-terms/#python-dictionary) only map from keys to values (not from values to keys). However, we can write a quick for loop to go the other direction. This cell is simply to demonstrate how the [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) is connected to the list entries in the [gensim](https://constellate.org/docs/key-terms/#gensim) ``bow_corpus``.

In [ ]:
# Find the token associated with a token id number
token_id = 100

# If the token id matches, print out the associated token
for dict_id, token in dictionary.items():
    if dict_id == token_id:
        print(token)

## Creating a Bag of Words Corpus

The next step is to connect our word frequency data found within ``documents`` to our [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) token IDs. For every document, we want to know how many times a word (notated by its ID) occurs. We will create a [Python list](https://constellate.org/docs/key-terms/#python-list) called ``bow_corpus`` that will turn our word counts into a series of [tuples](https://constellate.org/docs/key-terms/#tuple) where the first number is the [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) token ID and the second number is the word frequency.

![Combining Gensim dictionary with documents list to create Bag of Words Corpus](https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/bag-of-words-creation.png)

In [ ]:
# Create a bag of words corpus
bow_corpus = []

for document in documents:
    bow_corpus.append(dictionary.doc2bow(document))

print('Bag of words corpus created successfully.')

# The for loop could also be written as a list comprehension
# bow_corpus = [dictionary.doc2bow(document) for document in documents]

In [ ]:
# Examine the bag of words corpus for a specific document n
# Change the value of n to see another document
n = 100

list(bow_corpus[n][:25]) # List out a slice of the first 25 items

Using IDs can seem a little abstract, but we can discover the word associated with a particular ID. For demonstration purposes, the following code prints the associated token with the token counts. Each line printed below matches the token id and and count from above.

In [ ]:
# For each id and count in the bag of words corpus
# Print the corresponding word from the Gensim dictionary and count
for id, count in bow_corpus[n]:
    print(dictionary[id].ljust(15), count)

## Create the `TfidfModel`

The next step is to create the [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) model which will set the parameters for our implementation of [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf). In our [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) example, the formula for [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) was:

$$(Times-the-word-occurs-in-given-document) \cdot \mbox{log} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-word)}$$

In [gensim](https://constellate.org/docs/key-terms/#gensim), the default formula for measuring [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) uses log base 2 instead of log base 10, as shown:

$$(Times-the-word-occurs-in-given-document) \cdot \log_{2} \frac{(Total-number-of-documents)}{(Number-of-documents-containing-the-word)}$$

If you would like to use a different formula for your [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) calculation, there is a description of [parameters you can pass](https://radimrehurek.com/gensim/models/tfidfmodel.html).

In [ ]:
# Create our gensim TF-IDF model
model = gensim.models.TfidfModel(bow_corpus) 

Now, we apply our model to the ``bow_corpus`` to create our results in ``corpus_tfidf``. The ``corpus_tfidf`` is a python list of each document similar to ``bow_document``. Instead of listing the frequency next to the [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) ID, however, it contains the [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) score for the associated token. Below, we display the first document in ``corpus_tfidf``.

In [ ]:
# Create TF-IDF scores for the ``bow_corpus`` using our model
corpus_tfidf = model[bow_corpus]

In [ ]:
# List out the TF-IDF scores for the nth document's first 10 tokens
# Change n to change the document
n = 0

list(corpus_tfidf[n][:10])

Let's display the tokens instead of the [gensim dictionary](https://constellate.org/docs/key-terms/#gensim-dictionary) IDs.

In [ ]:
for id, score in corpus_tfidf[n][:10]:
    print(dictionary[id].ljust(20), score)

## Find Top Terms in a Single Document
Finally, let's sort the terms by their [TF-IDF](https://constellate.org/docs/key-terms/#tf-idf) weights to find the most significant terms in the document.

In [ ]:
# Sort the tuples in our tf-idf scores list

# Choosing a document by its index number
# Change n to see a different document
n = 0

def Sort(tfidf_tuples):
    "This sorts based on the second value in our tuple, the tf-idf score"
    tfidf_tuples.sort(key = lambda x: x[1], reverse=True)
    return tfidf_tuples 

# Print the document id and title
print('Title: ', document_titles[n])
print('ID: ', document_ids[n])
print('----------------------------------------')

# List the top twenty tokens in our example document by their TF-IDF scores
# First we sort the tokens with their scores
most_significant_terms = Sort(corpus_tfidf[n])[:20]

# Next we print the list, replacing the token ids with the tokens
for id, score in most_significant_terms:
    print(dictionary[id].ljust(20), score)

We could also analyze across the entire corpus to find the most unique terms. These are terms that appear in a particular text, but rarely or never appear in other texts. (Often, these will be proper names since a particular article may mention a name often but the name may rarely appear in other articles. There's also a fairly good chance these will be typos or errors in optical character recognition.)

In [ ]:
td = {}
for document in corpus_tfidf:
    for token_id, score in document:
        current_score = td.get(dictionary.get(token_id), 0)
        if current_score < score:
            td.update([(dictionary.get(token_id), score)])

In [ ]:
# Sort the items of ``td`` into a new variable ``sorted_td``
# the ``reverse`` starts from highest to lowest
sorted_td = sorted(td.items(), key=lambda kv: kv[1], reverse=True) 

for term, weight in sorted_td[:25]: # Print the top 25 terms in the entire corpus
    print(term, weight)

## Display Most Significant Term for each Document
We can see the most significant term in every document.

In [ ]:
# For each document, print the ID, most significant/unique word, and TF/IDF score

n = 0

for n, doc in enumerate(corpus_tfidf):
    if len(doc) < 1:
        continue
    word_id, score = max(doc, key=lambda x: x[1])
    print(document_ids[n], dictionary.get(word_id), score)
    if n >= 10:
        break

## Ranking documents by TF-IDF Score for a Search Word


In [ ]:
# Set a limit on the number of documents analyzed
limit = 500

terms_to_docs = defaultdict(list)
for doc_id, doc in enumerate(corpus_tfidf):
    for term_id, value in doc:
        term = dictionary.get(term_id)
        terms_to_docs[term].append((doc_id, value))
    if doc_id >= limit:
        break


In [ ]:
# Pick a unigram to discover its score across documents
search_term = 'coriolanus'

# Display a list of documents and scores for the search term

matching = terms_to_docs.get(search_term)

try: 
    for doc_id, score in sorted(matching, key=lambda x: x[1], reverse=True):
        print(document_ids[doc_id], score)
except:
    print('Search term not found. Change the term or expand the corpus size.')